In [1]:
import pandas as pd
import numpy as np
import os, sys, regex

In [2]:
dala = os.getenv('HOME')

In [3]:
base_dir = f'{dala}/Documents/RunSheets/AllRunSheets'

In [4]:
cols1 = ['Sample_Name', 'Reporter', 'CT', 'Ct_Mean', 'Well_Position']

data = pd.DataFrame(columns=cols1)
data_list = []
for file in os.listdir(base_dir):
    if file.endswith('.xls'):
        data_list.append(file)
    pass
sheets = sorted(data_list)

In [5]:
len(np.unique(np.array(sheets)))

499

In [6]:
data.head()

,Sample_Name,Reporter,CT,Ct_Mean,Well_Position


In [7]:
' '.join('2020-10-29 15:46:45 PM EAT'.split(' ')[:-2])

'2020-10-29 15:46:45'

In [8]:
pd.to_datetime(' '.join('2020-10-29 15:46:45 PM EAT'.split(' ')[:-2]), yearfirst=True)

Timestamp('2020-10-29 15:46:45')

In [9]:
cols2 = ['Sample Name', 'Reporter', 'CT', 'Ct Mean', 'Cт', 'Cт Mean', 'Well']
cols2b = ['Sample Name', 'Reporter', 'CT', 'Ct Mean', 'Cт', 'Cт Mean', 'Well Position']

counts = 0

for sheet in sheets:
    df = pd.read_excel(f'{base_dir}/{sheet}')
    count2 = 0
#     print(sheet)
    try:
        for row in df.itertuples():
            count2 += 1
            if row[1] == 'Experiment Run End Time':
                expt_run_date = pd.to_datetime(' '.join(row[2].split(' ')[:-2]), yearfirst=True)
            if row[1] == 'Well' and row[2] != 'Well Position':
                df = pd.read_excel(f'{base_dir}/{sheet}', 'Results', header=count2)
                cols3 = list(set(df.columns) & set(cols2))
    #             print(cols3)
                df2 = df[sorted(cols3)]
                df2.columns = sorted(cols1)
                df3 = df2.assign(Dt_Run=expt_run_date)
                data = data.append(df3, ignore_index=True)
                counts += 1
                break
            if row[1] == 'Well' and row[2] == 'Well Position':
                df = pd.read_excel(f'{base_dir}/{sheet}', 'Results', header=count2)
                cols3 = list(set(df.columns) & set(cols2b))
    #             print(cols3)
    #             print(list(df.columns))
                df2 = df[sorted(cols3)]
                df2.columns = sorted(cols1)
                df3 = df2.assign(Dt_Run=expt_run_date)
                data = data.append(df3, ignore_index=True)
                counts += 1
                break
    except:
        print(sheet)
        pass

In [10]:
print(f'number of sheets in folder: {len(sheets)}')
print(f'data rows: {data.shape[0]}')
print(f'sheets processed: {counts}')

number of sheets in folder: 499
data rows: 98816
sheets processed: 499


In [11]:
data.head()

,Sample_Name,Reporter,CT,Ct_Mean,Well_Position,Dt_Run
0,COVC 13541,FAM,Undetermined,NaN,A1,2020-10-12 15:33:47
1,COVC 13541,VIC,Undetermined,NaN,A1,2020-10-12 15:33:47
2,COVC 13541,CY5,21.914249,21.914249,A1,2020-10-12 15:33:47
3,COVC 13542,FAM,Undetermined,NaN,A2,2020-10-12 15:33:47
4,COVC 13542,VIC,Undetermined,NaN,A2,2020-10-12 15:33:47


In [12]:
data.info()#.nunique()#.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98816 entries, 0 to 98815
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Sample_Name    97334 non-null  object        
 1   Reporter       97535 non-null  object        
 2   CT             96956 non-null  object        
 3   Ct_Mean        45963 non-null  float64       
 4   Well_Position  98816 non-null  object        
 5   Dt_Run         97428 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 4.5+ MB


In [13]:
# undetermined_ids = list(data[data['CT'] == 'Undetermined']['Sample_Name'].unique())#.astype(list)

In [14]:
data2 = data[data['CT'] != 'Undetermined']

In [15]:
data3 = data2[data2['Reporter'].isin(['FAM', 'VIC', 'ABY', 'TAMRA']) == True]

In [16]:
data1 = data3[data3['Sample_Name'].isin(['PC', 'NC']) != True]

In [17]:
data4 = data1[data1.duplicated('Sample_Name', False) == True]

In [18]:
data4.head(10)

,Sample_Name,Reporter,CT,Ct_Mean,Well_Position,Dt_Run
126,COVC 13583,FAM,37.328472,37.328472,D7,2020-10-12 15:33:47
127,COVC 13583,VIC,42.683975,42.683975,D7,2020-10-12 15:33:47
135,COVC 13586,FAM,35.068436,35.068436,D10,2020-10-12 15:33:47
136,COVC 13586,VIC,39.385445,39.385445,D10,2020-10-12 15:33:47
186,COVC 13603,FAM,27.376181,27.376181,F3,2020-10-12 15:33:47
187,COVC 13603,VIC,31.060112,31.060112,F3,2020-10-12 15:33:47
219,COVC 13614,FAM,36.698231,36.698231,G2,2020-10-12 15:33:47
231,COVC 13618,FAM,32.132778,32.132778,G6,2020-10-12 15:33:47
232,COVC 13618,VIC,36.225323,36.225323,G6,2020-10-12 15:33:47
261,COVC 13628,FAM,33.411869,33.411869,H4,2020-10-12 15:33:47


In [19]:
def get_upper_alpha_seq():
    letters = []
    for c in range(65, 91):
        letters.append(chr(c))
    return letters

In [20]:
def cov_rename(x):
    try:
        if 'COV' not in x and '-R' in x:
            x = x.replace('-R', '')
        if 'COV' in x and 'COVC' not in x:
            x = x.replace('COV', 'COVC')
        int(x)
        x = str(x).replace(str(x), f'COVC{str(x)}')
    except (ValueError, AttributeError, TypeError): x = str(x).replace(' ', '')
    return x.replace('COVC_', 'COVC0')

In [21]:
def cov_names(x):
    cov_id = str(x).upper().replace(' ', '').rstrip(f'#-_{get_upper_alpha_seq()}')
    if len(cov_id) == 8:
        return cov_id.replace('COVC', 'COVC0')
    elif len(cov_id) == 7:
        return cov_id.replace('COVC', 'COVC00')
    elif len(cov_id) == 6:
        return cov_id.replace('COVC', 'COVC000')
    elif len(cov_id) == 5:
        return cov_id.replace('COVC', 'COVC0000')
    return cov_id

In [22]:
data4['Reporter'].unique()

array(['FAM', 'VIC', 'ABY', 'TAMRA'], dtype=object)

In [23]:
data5 = data4.assign(Sample_Name=data4['Sample_Name'].apply(lambda x: cov_names(cov_rename(x))))

In [24]:
data5.head()

,Sample_Name,Reporter,CT,Ct_Mean,Well_Position,Dt_Run
126,COVC13583,FAM,37.328472,37.328472,D7,2020-10-12 15:33:47
127,COVC13583,VIC,42.683975,42.683975,D7,2020-10-12 15:33:47
135,COVC13586,FAM,35.068436,35.068436,D10,2020-10-12 15:33:47
136,COVC13586,VIC,39.385445,39.385445,D10,2020-10-12 15:33:47
186,COVC13603,FAM,27.376181,27.376181,F3,2020-10-12 15:33:47


In [25]:
data6 = (data5.sort_values(['Sample_Name', 'Reporter', 'Dt_Run'])
 .drop_duplicates(['Sample_Name', 'Reporter'], keep='last', inplace=False))

In [26]:
data6#[data6['Sample_Name'].str.contains('COVC') == False]#['Dt_Run'].unique()

,Sample_Name,Reporter,CT,Ct_Mean,Well_Position,Dt_Run
37317,,ABY,22.004671,22.004671,H12,NaT
98546,,FAM,NaN,NaN,A7,NaT
98547,,VIC,NaN,NaN,A7,NaT
94554,A1,FAM,33.489597,31.697639,D1,2021-06-30 18:48:52
94555,A1,VIC,36.625538,34.432182,D1,2021-06-30 18:48:52
...,...,...,...,...,...,...
30332,SEWAGE2,VIC,37.116642,37.116642,H2,2021-04-08 15:47:49
27645,STD1_2X10^4,FAM,23.268881,24.886887,G1,2021-04-07 15:53:47
27633,STD3_2X10^2,FAM,31.191376,31.178442,F3,2021-04-07 15:53:47
27636,STD4_2X10^1,FAM,34.213322,33.965820,F4,2021-04-07 15:53:47


In [27]:
data7 = data6.sort_values(['Sample_Name']).groupby('Sample_Name').mean()

In [28]:
data7.head()

,Ct_Mean
Sample_Name,
,22.004671
A1,33.064911
C3,33.114729
COVC00137,37.646282
COVC00154,35.240269


In [29]:
data8 = data6.sort_values(['Sample_Name', 'Dt_Run']).drop_duplicates('Sample_Name', keep='last')#.to_frame()
data8

,Sample_Name,Reporter,CT,Ct_Mean,Well_Position,Dt_Run
98547,,VIC,NaN,NaN,A7,NaT
94555,A1,VIC,36.625538,34.432182,D1,2021-06-30 18:48:52
94561,C3,VIC,35.654247,34.974087,D3,2021-06-30 18:48:52
12797,COVC00137,VIC,39.514957,39.514957,D9,2020-06-03 17:04:05
12848,COVC00154,VIC,36.905937,36.905937,F2,2020-06-03 17:04:05
...,...,...,...,...,...,...
30332,SEWAGE2,VIC,37.116642,37.116642,H2,2021-04-08 15:47:49
27645,STD1_2X10^4,FAM,23.268881,24.886887,G1,2021-04-07 15:53:47
27633,STD3_2X10^2,FAM,31.191376,31.178442,F3,2021-04-07 15:53:47
27636,STD4_2X10^1,FAM,34.213322,33.965820,F4,2021-04-07 15:53:47


In [30]:
data9 = data8[['Sample_Name', 'Well_Position', 'Dt_Run']].merge(data7, how='right', left_on='Sample_Name', right_index=True)

In [31]:
df_cts = data9[data9['Sample_Name'].str.contains('COV') == True]

In [32]:
df_cts.sort_values('Dt_Run').to_excel('/home/douso/Documents/RunSheets/Cts/runsheet-cts.xlsx', index=False, float_format='%.1f')